# Deeper Dive: DSP & Python

![](assets/transitions/lab_trans_dsp_python.svg)

> In this notebook we'll introduce some development tools for digital signal processing (DSP) using Python and JupyterLab. Unlike the previous notebooks, there is absolutely no hidden code here. We're showing you the complete development story behind an example DSP application.
>
> In our example application, we'll start by visualising some interesting signals — audio recordings of Scottish birds! We'll then use a few different analytical techniques to gain some understanding of these signals and finally process the audio to isolate a single type of bird.

## Inspecting our signal

In the assets folder there is an audio file, `birds.wav`. This was recorded by Stuart Fisher and released under [CC BY-NC-ND 2.5](https://creativecommons.org/licenses/by-nc-nd/2.5/); accessible [here](https://www.xeno-canto.org/28039).

Before we get into our signal processing at all, let's give it a listen. We can do that through our browser using IPython's rich set of display functions.

In [ ]:
from IPython.display import Audio
Audio("assets/pdd/birds.wav")

OK, so what are we hearing? We've got two main subjects here:
  1. The lower pitched bird (going "cuurloo!") is a eurasian curlew
  2. The higher pitched bird chatting away is a chaffinch

Just for context, here's what these birds look like:
<div style='max-width: 1005px;'>
<div style='width:45%; float:left; text-align:center;'>
    <img src="assets/pdd/curlew.jpg"/>
    <b>Curlew</b> <br/>Photo by Vedant Raju Kasambe <br/> <a href="https://creativecommons.org/licenses/by-sa/4.0/deed.en">Creative Commons Attribution-Share Alike 4.0</a>
</div>
<div style='width:45%; float:right; text-align:center;'>
    <img src="assets/pdd/chaffinch.jpg"/>
    <b>Chaffinch</b> <br/>Photo by Charles J Sharp <br/> <a href="https://creativecommons.org/licenses/by/3.0/deed.en">Creative Commons Attribution 3.0</a>
</div>
</div>

### Loading from disk

Let's get this audio file loaded in Python so we can perform some visualisation. We're going to make use of the [SciPy](https://www.scipy.org/) ecosystem for most of our signal processing in Python. To load the `.wav` file in to our environment as an array of samples, let's use SciPy's `wavfile` IO module.

In [ ]:
from scipy.io import wavfile

fs, aud_in = wavfile.read("assets/pdd/birds.wav")

`wavfile.read` gives us two things: the sampling frequency of the signal (`fs`), and the raw samples as an array (`aud_in`). Let's check the sampling frequency.

In [ ]:
fs

The sampling frequency of the recording is 44.1 kHz — the standard rate for CD quality audio. Now let's look at the format of the samples themselves. To start, what is the type of our sample array? 

In [ ]:
type(aud_in)

This is an N-dimensional array ('ndarray') from the NumPy package, that you'll remember from the introduction notebook.

Let's interrogate this array a little futher. We should be aware of its length and the data type of each element:

In [ ]:
len(aud_in)

In [ ]:
aud_in.dtype

So each sample is a signed 16 bit integer, and we have over half a million samples in total! We can comfortably fit this in memory (it's just over 1 MB) but we will need to do some processing to visualise all of this data in a useful format.

### Plotting in the time domain

As a first investigation, let's plot only a short clip from the recording. We'll use [plotly_express](https://www.plotly.express/) here because it generates impressive, interactive plots with surprisingly small amounts of code. `plotly_express` expects input data to be given as a [pandas data frame](http://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html#overview), so we'll need to do a little bit of conversion work upfront. We build up a frame with multiple columns (time and amplitude, in this case) and then we can efficiently traverse, sort, and search the data. 

In [ ]:
import pandas as pd
import numpy as np

def to_time_dataframe(samples, fs):
    """Create a pandas dataframe from an ndarray of 16-bit time domain samples"""
    num_samples = len(samples)
    sample_times = np.linspace(0, num_samples/fs, num_samples)
    normalised_samples = samples / 2**15
    return pd.DataFrame(dict(
        amplitude = normalised_samples,  
        time      = sample_times
    ))

Now that we can turn our sample array into a data frame, let's pass it to `plotly_express` to create a simple, time-domain plot. First let's make a base template for our plots.

In [ ]:
# Derive a custom plotting template from `plotly`
import plotly.io as pio
new_template = pio.templates['plotly']
new_template.update(dict(layout = dict(
        width         = 800,
        autosize      = False,
        legend        = dict(x=1.1),
)))

# Register new template as the default
pio.templates['dsp_plot'] = new_template
pio.templates.default = 'dsp_plot'

Now we can get plotly to plot a snippet of the audio, and it will be in the theme we described above.

In [ ]:
import plotly_express as px

# Let's take a small subset of the recording
aud_clip = to_time_dataframe(aud_in, fs).query('0.3 < time < 0.718')

# Plot signal
px.line(                                                             # Make a line plot with...
    aud_clip,                                                        #   Data frame
    x='time', y='amplitude',                                         #   Axes field names
    labels = dict(amplitude='Normalised Amplitude', time='Time (s)') #   Axes label names
)

This plot is interactive. Feel free to zoom in (click and drag) and pan around. You should be able to zoom in far enough to see the single sinusoidal cycles. Double click anywhere on the plot to zoom back out.

There is clearly some activity in this waveform, but it's hard to imagine what this should sound like from the time domain alone. Sure we can get a feel for the volume of the signal over time, but what are the different pitches/frequencies in this sound? Let's take a look at the same snippet in the frequency domain to find out.

### Plotting in the frequency domain

We can use SciPy to perform a Fast Fourier Transform (FFT) to convert our time domain signal into the frequency domain. The `fft` function performs an FFT for our input. Let's try this out on the small audio clip from above.

In [ ]:
from scipy.fftpack import fft

def to_freq_dataframe(samples, fs):
    """Create a pandas dataframe from an ndarray frequency domain samples"""
    sample_freqs = np.linspace(0, fs, len(samples))
    return pd.DataFrame(dict(
        amplitude = samples[0:int(len(samples)/2)],  
        freq      = sample_freqs[0:int(len(samples)/2)]
    ))

# Take slice of full input
aud_clip_numpy = aud_in[int(0.3*fs): int(0.718*fs)] 

# Perform FFT
NFFT = 2**14 # use a generous length here for maximum resolution
aud_clip_fft = np.abs(fft(aud_clip_numpy,NFFT))

# Plot FFT
px.line(
    to_freq_dataframe(aud_clip_fft, fs),
    x='freq', y='amplitude',
    labels = dict(amplitude='Amplitude', freq='Freq (Hz)')
)

There are a couple of features to note in the frequency domain that we had totally missed in the time domain:

  1. *What a generous sampling rate!*
     
     As the original sample rate is 44.1 kHz, the recording is able to represent any frequencies up to 22 kHz. However, there are no significant frequency components above 5 kHz so we could resample this signal to have about $\frac{1}{3}$ of the data and still retain almost all of the useful information. This should speed up calculations and reduce memory requirements.
     
  2. *Bird identification!*
  
     There are two clear and distinct signals: one at $\approx$ 1.7 kHz and one at $\approx$ 4 kHz. Go back and see just how difficult this is to identify in the time domain.
     
     The lower frequency signal is from the curlew and the higher frequency is from the chaffinch. There is also some faint noise under 50 Hz from wind picked up by the microphone. It should be possible to employ some filtering to completely isolate one bird's sound from the other, but we'll get back to this later on in the notebook.
     
We've been able to glean more of an understanding of the signal's composition by using SciPy to view it the frequency domain. There's one final visualisation tool that we should employ here moving on — the spectrogram!

### Plotting as a spectrogram

The spectrogram can essentially give us a simultaneous view of both time and frequency by plotting how the FFT of the signal varies with time, with a spectrum of colours to represent signal amplitude.

These plots are a little more advanced, so we move away from `plotly_express` and use a lower-level plotly API.

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
from scipy.signal import spectrogram, decimate

def plot_spectrogram(samples, fs, decimation_factor=3, max_heat=50, mode='2D'):
    
    # Optionally decimate input
    if decimation_factor>1:
        samples_dec = decimate(samples, decimation_factor, zero_phase=True)
        fs_dec = int(fs / decimation_factor)
    else:
        samples_dec = samples
        fs_dec = fs

    # Calculate spectrogram (an array of FFTs from small windows of our signal)
    f_label, t_label, spec_data = spectrogram(
        samples_dec, fs=fs_dec, mode="magnitude"
    )
    
    # Make a plotly heatmap/surface graph
    layout = go.Layout(
        height=500,
        # 2D axis titles
        xaxis=dict(title='Time (s)'),
        yaxis=dict(title='Frequency (Hz)'),
        # 3D axis titles
        scene=dict(
            xaxis=dict(title='Time (s)'),
            yaxis=dict(title='Frequency (Hz)'),
            zaxis=dict(title='Amplitude')
        )
    )
    
    trace = go.Heatmap(
        z=np.clip(spec_data,0,max_heat),
        y=f_label,
        x=t_label
    ) if mode=='2D' else go.Surface(
        z=spec_data,
        y=f_label,
        x=t_label
    )
    
    py.iplot(dict(data=[trace], layout=layout))


plot_spectrogram(aud_in, fs, mode='2D')

Again, we can see the two bird noises quite distinctly — the curlew between 1.2 $\rightarrow$ 2.6 kHz and the chaffinch between 3 $\rightarrow$ 5 kHz. This time, however, we can see how these sounds change over time. The curlew has a smooth sweeping call followed by a short, constant tone while the chaffinch produces a more erratic spectrogram as it jumps between tones in quick succession.

Next we'll look at designing some filters from Python so we can isolate one of the birds.

## FIR filtering

We can use functions from SciPy's signal module to design some FIR filter coefficients and perform the filtering:

  * `firwin` can design filter weights that meet a given spec — cut off frequencies, ripple, filter type...
  * `freqz` helps us calculate the frequency response of the filter. Useful for checking the characteristics of the generated filter weights.
  * `lfilter` actually performs the filtering of our signal.
  
>If you have used MATLAB these functions will feel familiar to you. One thing to note though is, unlike MATLAB, arrays (or lists) in Python are zero-indexed and array elements are referenced by square brackets, rather than parentheses.

### High-pass filter for chaffinch isolation

Let's start by designing a filter to isolate the chaffinch sounds. This should be a high-pass filter with the aim of suppressing all signals below 2.6 kHz approximately. To give ourselves some breathing space, we should ask for a filter with a cutoff frequency a little higher than 2.6 kHz; let's say 2.8 kHz.

In [ ]:
from scipy.signal import freqz, firwin

nyq = fs / 2.0
taps = 99

# Design high-pass filter with cut-off at 2.8 kHz
hpf_coeffs = firwin(taps, 2800/nyq, pass_zero=False)

def plot_fir_response(coeffs, fs):
    """Plot the frequency magnitude response of a set of FIR filter weights"""
    
    freqs, resp = freqz(coeffs, 1)
    return px.line(
        to_freq_dataframe(np.abs(resp), fs/2),
        x='freq', y='amplitude',
        labels = dict(amplitude='Normalised amplitude', freq='Freq (Hz)')
    )

# We'll also be using these coefficients in the next notebook so let's save them to a file for later...
np.save('assets/pdd/hpf_coeffs.npy', hpf_coeffs)

# Plot our filter's frequency response as a sanity check
plot_fir_response(hpf_coeffs, fs)

So, we asked for a cut-off frequency of 2.8 kHz and we can use the cursor with the plot above to verify this. Hover over the trace at $\approx$0.5 amplitude and it should report that this point corresponds to 2.8 kHz.

Now it's time to use these filter coefficients to filter the original audio! Let's do this in software with `lfilter` just now, plot the resulting spectrogram, and save a `.wav` file for playback.

In [ ]:
from scipy.signal import lfilter

# Filter audio
aud_hpf = lfilter(hpf_coeffs, 1.0, aud_in)

# Plot filtered audio
plot_spectrogram(aud_hpf, fs)

# Offer audio widget to hear filtered audio
wavfile.write('assets/pdd/hpf.wav', fs, np.array(aud_hpf, dtype=np.int16))
Audio('assets/pdd/hpf.wav')

Hopefully we can confirm both visually and aurally that we've isolated the chaffinch sounds from the curlew and the wind. Sounds pretty good! 

>It is also possible to isolate the curlew, this time with a bandpass filter. If time permits, design and implement the filter using the techniques we've covered above and plot the results (check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html) for the `firwin` function if you need help)

## Summary 

We've reached the end of our first *Deeper Dive* notebook, so let's quickly recap what we've covered:

  * Using the JupyterLab and Python environment as a DSP prototyping platform:
    + Introducing the SciPy ecosystem, including the `scipy.signal` module for DSP operations and `numpy` for efficient arrays.
    + Visualisation with `plotly_express` and `pandas` data frames
  * Using Python to inspect signals in the time and frequency domains
  * Designing FIR filters with SciPy and verifying their frequency responses
  * Performing FIR filtering in software
      
In the next *Deeper Dive* we will use the techniques learned here to interact with DSP IP on the FPGA. Using the power of PYNQ, we will then control this hardware directly from the notebook!

[⬅️ Previous](03_pynq_and_sd_fec.ipynb) 👩‍💻  [Next ➡️](05_dsp_and_pynq.ipynb)